In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0,'../')

In [ ]:
from mllib.nlp.datasets.cmudict import CMUDict
from datasets import load_dataset
from mllib.nlp.seq2seq import Seq2Seq

In [ ]:
ds = load_dataset('/notebooks/dlnotebooks/mllib/nlp/datasets/cmudict.py')

Downloading:   0%|          | 0.00/919k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cmu_dict downloaded and prepared to /root/.cache/huggingface/datasets/cmu_dict/cmu3/1.0.0/a0e598136ef9603a0d6d97059f5e1d2cac789cfe3c0998cb2b4b7fd4198da504. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_test = ds['train'].train_test_split(test_size=0.2)

In [ ]:
train_test['train'][0]

{'word': 'preempted',
 'word_length': 9,
 'phoneme': ['P', 'R', 'IY0', 'EH1', 'M', 'P', 'T', 'IH0', 'D']}

# Data Processing

Processing data with map inspired by `tf.dataset.map` map method 

In [ ]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator



In [ ]:
train_test['train']['word'][0], train_test['train']['phoneme'][0] 

('preempted', ['P', 'R', 'IY0', 'EH1', 'M', 'P', 'T', 'IH0', 'D'])

In [ ]:
build_vocab_from_iterator??

Signature:
build_vocab_from_iterator(
    iterator: Iterable,
    min_freq: int = 1,
    specials: Union[List[str], NoneType] = None,
    special_first: bool = True,
) -> torchtext.vocab.vocab.Vocab
Source:   
def build_vocab_from_iterator(iterator: Iterable, min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> Vocab:
    """
    Build a Vocab from an iterator.

    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        min_freq: The minimum frequency needed to include a token in the vocabulary.
        specials: Special symbols to add. The order of supplied tokens will be preserved.
        special_first: Indicates whether to insert symbols at the beginning or at the end.


    Returns:
        torchtext.vocab.Vocab: A `Vocab` object

    Examples:
        >>> #generating vocab from text file
        >>> import io
        >>> from torchtext.vocab import build_vocab_from_iterator
        >>> def yield_toke

In [ ]:
phoneme_vocab = build_vocab_from_iterator(train_test['train']['phoneme'],specials=['<unk>','<sos>'])
word_vocab = build_vocab_from_iterator(train_test['train']['word'],specials=['<unk>'])

In [ ]:
phoneme_vocab(['<unk>','<sos>'])

[0, 1]

In [ ]:
phoneme_vocab.set_default_index(0)
word_vocab.set_default_index(0)

In [ ]:
word_vocab.lookup_indices(['a','b','c'])

[2, 17, 10]

# Data Collator

In [ ]:
BATCH_SIZE = 8

In [ ]:
import numpy as np

def process_single_example(word_tokens, phoneme, word_length):
    # Heree you can add variety of operations, Not only is it tokenize
    # The object that this function handles, Namely dataset this data type, adopt featuer
    
    src = word_vocab.lookup_indices(list(word_tokens))
    trg = phoneme_vocab.lookup_indices(phoneme)

    return src, trg, word_length

def collate_batch(batch):
    
    batch_size = len(batch['word'])
    out = [process_single_example(*tokens) for tokens in zip(batch['word'], batch['phoneme'], batch['word_length'])]
    

    return {
        'src': [b[0] for b in out],
        'trg': [b[1] for b in out],
        'src_len': [b[2] for b in out],
    }

In [ ]:
ds_processed = train_test.map(collate_batch, remove_columns=['word','word_length','phoneme'], 
                        batch_size= BATCH_SIZE,
                           batched=True).with_format('pytorch', output_all_columns=True)

  0%|          | 0/13516 [00:00<?, ?ba/s]

  0%|          | 0/3379 [00:00<?, ?ba/s]

In [ ]:
ds_processed['train'][2]

{'src': tensor([17,  7,  8,  8,  4,  6, 15]),
 'trg': tensor([15, 19,  6, 10, 25]),
 'src_len': tensor(7)}

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def pad_collate(batch):
    
    def pad(xs):
        return torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)
    
    src = [b['src'] for b in batch]
    trg = [b['trg'] for b in batch]
    src_len = [b['src_len'] for b in batch]
    
    return {
        'src': pad(src), 
        'trg' : pad(trg), 
        'src_len' : src_len
    }


In [ ]:
dls = DataLoader(ds_processed['train'], shuffle=True, collate_fn=pad_collate, batch_size=32)

In [ ]:
#next(iter(dls))

In [ ]:
from transformers import DataCollatorWithPadding, default_data_collator

In [ ]:
import random
# checking
def decode_word(lst):
    return ''.join(word_vocab.lookup_tokens(lst.numpy()))

def decode_phoneme(lst):
    return ','.join(phoneme_vocab.lookup_tokens(lst.numpy()))

indices = random.sample(range(10,1000), 5 )

for l in indices:
    src = decode_word(ds_processed['train']['src'][l])
    trg = decode_phoneme(ds_processed['train']['trg'][l])
    src_len = ds_processed['train']['src_len'][l]
    print(src, trg, src_len)

benigno B,EH2,N,IY1,N,Y,OW0 tensor(7)
canilles K,AH0,N,IH1,L,IY0,Z tensor(8)
hals HH,AE1,L,Z tensor(4)
pennella P,EH2,N,EH1,L,AH0 tensor(8)
kardos K,AA1,R,D,OW0,Z tensor(6)


# Model building

In [ ]:
import pytorch_lightning as pl

In [ ]:
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger


In [ ]:
lr_monitor = LearningRateMonitor(logging_interval='step')
logger = TensorBoardLogger('tb_logs', name='my_model')
trainer = pl.Trainer(callbacks=[lr_monitor],max_epochs=1, gpus=1, logger=[logger])

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
input_vocab_size = len(word_vocab)
output_vocab_size  = len(phoneme_vocab)

model = Seq2Seq(input_vocab_size, output_vocab_size,p=0.1)

In [ ]:
batch = next(iter(dls))

In [ ]:
trainer.fit(model, train_dataloader=dls)


  | Name    | Type             | Params
---------------------------------------------
0 | _loss   | CrossEntropyLoss | 0     
1 | encoder | Encoder          | 14.0 K
2 | decoder | Decoder          | 17.3 K
---------------------------------------------
31.3 K    Trainable params
0         Non-trainable params
31.3 K    Total params
0.125     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 1it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1